In [52]:
# ROME (you implemented)
# SERAC
# KE
# SLAG
# MALMEN
# GRACE

SERAC MODEL Workflow -
<Br>
->Base model whose parameters are to be frozen (in this we have taken llama 3.2-1B model)
<Br>
->Scope Classifier (gϕ), which decides if a new user query matches any edit from memory
<Br>
->Counterfactual Model, which gives the edited response if the query matches any edit

In [1]:
!rm -rf /root/.cache/huggingface
!rm -rf /root/.cache/transformers


In [2]:
!pip install -q transformers accelerate datasets sentence-transformers faiss-cpu peft bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 66.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 27.9 MB/s eta 0:00:00:00:0100:01


In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_TOKEN")
import os
from huggingface_hub import login

login(token=secret_value_0)



In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "meta-llama/Llama-3.2-1B"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_auth_token=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=True,          # ⭐ important
    torch_dtype=torch.float16,
    device_map="auto"
)
model.eval()
print("Model loaded.")

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/146 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Model loaded.


In [7]:
tokenizer.pad_token = tokenizer.eos_token


In [8]:
!wget https://rome.baulab.info/data/dsets/counterfact.json

--2026-02-20 01:38:59--  https://rome.baulab.info/data/dsets/counterfact.json
Resolving rome.baulab.info (rome.baulab.info)... 35.232.255.106
Connecting to rome.baulab.info (rome.baulab.info)|35.232.255.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45108470 (43M) [application/json]
Saving to: ‘counterfact.json’

counterfact.json    100%[===================>]  43.02M  63.7MB/s    in 0.7s    

2026-02-20 01:39:00 (63.7 MB/s) - ‘counterfact.json’ saved [45108470/45108470]



In [9]:
import json

with open("counterfact.json", "r") as f:
    counterfact = json.load(f)

print("Total examples:", len(counterfact))

Total examples: 21919


In [10]:
counterfact[0]

{'case_id': 0,
 'pararel_idx': 2796,
 'requested_rewrite': {'prompt': 'The mother tongue of {} is',
  'relation_id': 'P103',
  'target_new': {'str': 'English', 'id': 'Q1860'},
  'target_true': {'str': 'French', 'id': 'Q150'},
  'subject': 'Danielle Darrieux'},
 'paraphrase_prompts': ['Shayna does this and Yossel goes still and dies. Danielle Darrieux, a native',
  'An album was recorded for Capitol Nashville but never released. Danielle Darrieux spoke the language'],
 'neighborhood_prompts': ['The mother tongue of Léon Blum is',
  'The native language of Montesquieu is',
  'François Bayrou, a native',
  'The native language of Raymond Barre is',
  'Michel Rocard is a native speaker of',
  'Jacques Chaban-Delmas is a native speaker of',
  'The native language of François Bayrou is',
  'Maurice Genevoix, speaker of',
  'The mother tongue of François Bayrou is',
  'Melchior de Vogüé, speaker of'],
 'attribute_prompts': ['J.\xa0R.\xa0R. Tolkien is a native speaker of',
  'The mother tongue

In [11]:
ex = counterfact[0]

In [12]:
def extract_counterfact(example):
    """
    General extractor for CounterFact-style datasets.
    Returns a dict usable by SERAC pipeline.
    """

    rw = example.get("requested_rewrite", {})

    subject = rw.get("subject")

    prompt_template = rw.get("prompt", "")

    # Building final prompt safely
    if subject is not None and "{}" in prompt_template:
        prompt = prompt_template.format(subject)
    else:
        prompt = prompt_template

    target_new = rw.get("target_new", {}).get("str")
    target_true = rw.get("target_true", {}).get("str")

    return {
        "prompt": prompt,
        "subject": subject,
        "target_new": target_new,
        "target_true": target_true,
        "paraphrases": example.get("paraphrase_prompts", []),
        "neighborhood": example.get("neighborhood_prompts", []),
        "generation": example.get("generation_prompts", []),
        "attribute": example.get("attribute_prompts", [])
    }


In [13]:
import torch

class SeracMemoryLlama:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

        self.prompts = []
        self.targets = []

        self.embeddings = None   

    
    def embed(self, texts):
        inputs = self.tokenizer(
            texts,
            return_tensors="pt",
            padding=True,
            truncation=True
        ).to(self.model.device)  # converts the words to tokens

        with torch.no_grad():
            out = self.model(**inputs, output_hidden_states=True)

        hidden = out.hidden_states[-1]              # (B, T, D) and from that get the vector representation of last layer(-1)
        mask = inputs["attention_mask"].unsqueeze(-1)  # expanding the attention mask from (B, T) TO (B, T, 1)

        hidden = hidden * mask # TO REMOVE THE influence of the padded tokens
        emb = hidden.sum(dim=1) / mask.sum(dim=1)   # masked mean

        emb = torch.nn.functional.normalize(emb, dim=-1) # averaged to get the sentence meaning in a single vector

        return emb.cpu()

    def add_edit(self, prompt, target, subject):
        if prompt is None or target is None:
            return
    
        self.prompts.append(prompt)
        self.targets.append(target)
    
        key = f"{subject} || {prompt}"  # introduced the subject here to increase the similarity for the subject for which the answer is to be changed   
    
        emb = self.embed([key]) # make the embedding vector for the subject and prompt
    
        if self.embeddings is None:
            self.embeddings = emb
        else:
            self.embeddings = torch.cat([self.embeddings, emb], dim=0) # to stack embeddings for more number of edits 


    def retrieve(self, query, subject=None, threshold=0.85):
        if self.embeddings is None:
            return None
    
        if subject is not None:
            key = f"{subject} || {query}" 
        else:
            key = query
    
        q = self.embed([key])
    
        sims = torch.matmul(self.embeddings, q.T).squeeze(-1) 
        # E. q to find the similarity between the query and all edits to find if there is any edit similar to query for which answer is to be changed
    
        idx = torch.argmax(sims).item() 
        score = sims[idx].item() #similarity score
    
        if score > threshold: 
            return self.targets[idx]
    
        return None


In [14]:
memory = SeracMemoryLlama(model, tokenizer)

edit = extract_counterfact(ex)

memory.add_edit(edit["prompt"], edit["target_new"], edit["subject"])



In [15]:
print(edit)

{'prompt': 'The mother tongue of Danielle Darrieux is', 'subject': 'Danielle Darrieux', 'target_new': 'English', 'target_true': 'French', 'paraphrases': ['Shayna does this and Yossel goes still and dies. Danielle Darrieux, a native', 'An album was recorded for Capitol Nashville but never released. Danielle Darrieux spoke the language'], 'neighborhood': ['The mother tongue of Léon Blum is', 'The native language of Montesquieu is', 'François Bayrou, a native', 'The native language of Raymond Barre is', 'Michel Rocard is a native speaker of', 'Jacques Chaban-Delmas is a native speaker of', 'The native language of François Bayrou is', 'Maurice Genevoix, speaker of', 'The mother tongue of François Bayrou is', 'Melchior de Vogüé, speaker of'], 'generation': ["Danielle Darrieux's mother tongue is", 'Where Danielle Darrieux is from, people speak the language of', "Danielle Darrieux's mother tongue is", 'Danielle Darrieux was born in', "Danielle Darrieux's mother tongue is", "Danielle Darrieux'

In [16]:
def serac_generate(query, memory, model, tokenizer, subject=edit["subject"]):
    retrieved = memory.retrieve(query)

    # handle (target, score) vs target
    if isinstance(retrieved, tuple):
        retrieved = retrieved[0]

    if retrieved is not None:
        query = f"{query} {retrieved}"

    inputs = tokenizer(query, return_tensors="pt").to(model.device)

    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=40)

    return tokenizer.decode(out[0], skip_special_tokens=True)

In [17]:
prompt = edit["prompt"]

print("BASE:")
print(serac_generate(prompt, SeracMemoryLlama(model, tokenizer), model, tokenizer))

print("\nAFTER EDIT:")
print(serac_generate(prompt, memory, model, tokenizer))


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


BASE:


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The mother tongue of Danielle Darrieux is French, but she is not French. She is a French-Algerian woman. She was born in Algiers in 1903. She is the daughter of a French diplomat, and her father

AFTER EDIT:
The mother tongue of Danielle Darrieux is English. She has been married to the actor and director Jean-Pierre Aumont since 1950 and has one daughter, the actress Anne Parillaud. She was born in 1902 in Paris


In [18]:
for p in edit["paraphrases"][:5]:
    print("\nPrompt:", p)
    print(serac_generate(p, memory, model, tokenizer))


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Prompt: Shayna does this and Yossel goes still and dies. Danielle Darrieux, a native


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Shayna does this and Yossel goes still and dies. Danielle Darrieux, a native English speaker, is a film actress and stage actress. She was a member of the original Broadway cast of Follies in 1971, and the film version of the musical in 1971.


Prompt: An album was recorded for Capitol Nashville but never released. Danielle Darrieux spoke the language
An album was recorded for Capitol Nashville but never released. Danielle Darrieux spoke the language of her heart and soul. This is a must-have for fans of the singer/songwriter.
Danielle Darrieux (born 1944) was born in Paris, France. She began


In [19]:
for p in edit["neighborhood"][:5]:
    print("\nNeighborhood:", p)
    print(serac_generate(p, memory, model, tokenizer))


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Neighborhood: The mother tongue of Léon Blum is


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The mother tongue of Léon Blum is English, but his mother tongue is French. This is the case for many French people. For example, in 2011, the French Ministry of Education estimated that 17% of French people were bilingual

Neighborhood: The native language of Montesquieu is


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The native language of Montesquieu is English. He was born in 1689 in Bordeaux, France.
He was a French writer and statesman, best known for his book The Spirit of the Laws.
He was a great influence on the

Neighborhood: François Bayrou, a native


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


François Bayrou, a native of France, is a French politician who has served as the leader of the centre-right Union for French Democracy (UDF) since 2002 and as a member of the National Assembly since 199

Neighborhood: The native language of Raymond Barre is


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The native language of Raymond Barre is English. He is a writer, a politician, a philosopher and a social critic. He is the founder of the Centre de Sociologie de l’Action Publique, a research centre in Paris. He

Neighborhood: Michel Rocard is a native speaker of
Michel Rocard is a native speaker of English and French. He was born in 1940 in Paris. He received his B.A. in Philosophy and Letters in 1964 from the University of Paris. In 1968 he received his


In [20]:
def contains_answer(text, answer):
    return answer.lower() in text.lower()


In [21]:
def evaluate_example(example, memory, model, tokenizer):
    edit = extract_counterfact(example)

    # add edit to memory
    memory.add_edit(edit["prompt"], edit["target_new"], edit["subject"])

    # -------- EDIT SCORE --------
    out = serac_generate(edit["prompt"], memory, model, tokenizer)
    edit_score = int(contains_answer(out, edit["target_new"]))

    # -------- PARAPHRASE SCORE --------
    para_scores = []
    for p in edit["paraphrases"]:
        out = serac_generate(p, memory, model, tokenizer, subject=edit["subject"])
        para_scores.append(int(contains_answer(out, edit["target_new"])))

    paraphrase_score = sum(para_scores) / max(len(para_scores), 1)

    # -------- NEIGHBORHOOD SCORE --------
    neigh_scores = []
    for p in edit["neighborhood"]:
        out = serac_generate(p, memory, model, tokenizer, subject=None)

        # good if DOES NOT output new target
        neigh_scores.append(int(not contains_answer(out, edit["target_new"])))

    neighborhood_score = sum(neigh_scores) / max(len(neigh_scores), 1)

    return edit_score, paraphrase_score, neighborhood_score


In [23]:
memory = SeracMemoryLlama(model, tokenizer)

results = []
es_f = 0.00
ps_f = 0.00
ns_f = 0.00

for i in range(20):
    ex = counterfact[i]

    es, ps, ns = evaluate_example(ex, memory, model, tokenizer)

    results.append({
        "idx": i,
        "edit": es,
        "paraphrase": ps,
        "neighborhood": ns
    })
    es_f += es
    ps_f += ps
    ns_f += ns

    print(i, es, ps, ns)
print(es_f/(i+1), ps_f/(i+1), ns_f/(i+1))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

0 1 0.5 0.2


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


1 1 0.5 0.5


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

17 1 1.0 0.3


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


18 1 0.0 0.0


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


19 1 1.0 0.9
1.0 0.575 0.49000000000000005
